# Bayesian Data Analysis Project: Gene Cluster Analysis Using Single cell RNA-sequencing

In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import gzip
import cPickle as cpkl
import seaborn as sns
import pandas as pd

/Users/casperkaae/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Introduction

Single-cell RNA sequencing measures RNA-expression levels for individual cells allowing identification of RNA-expression in cell subpopulations. The goal of this study is to cluster the single cell expression profiles into clusters using probilistic modelling and see if these resemblance clusters produced by a skilled bioinformatician.

The clustering produced by the bioinformatician is seen below. Some clustering can be seen, however the data also appears relatively noisy

<img src="figures/bioinf_clusters.png",width=800>
### Dataset

The data is obtained from single-cell RNA sequencing of bone marrow from mice. The reads were mapped to the mice genome and each read mapping to a gene were measured as a *count* for the corresponding gene. Data were measured for for 1500 samples and ≈27000 genes. We selected a subset of 48 interesting genes for furhter study based on a previous  litterature study. Secondly we were also provided with cell-type labels calculated using the method presented in [Franziska et. al. 2015].



#### Dataset analysis
Due to low amounts of RNA in a single living cell, the dataset is very sparse with 95% of the genes (in the filtered dataset) having a count of zero and approximately 5.5 counts per sample on average.

<img src="figures/histogram.png",width=400>

Using the TSNE dimensionality reduction algorithm we visualized the complete dataset (27000 genes) and the filtered dataset (48 genes) as seen below. The non-filtered dataset have almost no structure whereas some structure is seen for the filtered dataset. Secondly we also see that the computed cell-type labels show no agreement with the structure in the filtered dataset suggesting that these should not be trusted.
<img src="figures/tsne_all.png",width=400>
<img src="figures/tsne_filtered.png",width=400>






## Methods and Results

The latent Dirichlet allocation model (LDA) by [Blei et al. 2003] can be seen as an unsupervised model for cluster analysis. LDA is a generative model that from a *hidden* representation optimizes its posterior in order to explain the observed data points. The observed data is perceived as continues count data where each feature is expected to be exchangeable. The model is referred to as a topic model, since it segments the input data points into a latent topic distribution, which is a multinomial representation of the probability of an input data point being in each of *K* topics. The segmentation into topics, can be perceived as clusters in a $k$-dimensional output space (e.g. euclidian) and be analyzed with various similarity/distance measurements.

To explain each variable of the LDA model we use the terminology of topic modeling for documents. In the model we have (i) $\alpha$ Dirichlet prior over the topic distribution of each document, (ii) $\beta$ Dirichlet prior over the word distribution for each topic, (iii) $\theta_i$ the multinomial topic distribution for a document $i$ with length $N$, (iv) $\rho_k$ the word distribution for a topic $k$, (v) $z^k_{ij}$ the topic $k$ for the $i^{th}$ document and the $j^{th}$ word, (vi) $w_{ij}$ is the $j^{th}$ word in document $i$. The plate notation for the model is:

<img src="figures/lda.png",width=600>

From [Blei et al. 2003] we find that the joint probility for $\theta_i$ and $z^k_{ij}$ and $w_{ij}$ is given by:
$$
p(\theta, z, w|\alpha, \beta) = p(\theta|\alpha) \Pi_{n=1}^{N} p(z_n|\theta)p(w_n|z_n,\beta) \ .
$$

The corresponding probability of a word $w_{ij}$ is given by (please note the infeasible integral over $\theta$):

$$
p(w|\alpha, \beta) = \Pi_{d=1}^M \int p(\theta_d|\alpha) \bigg (\Pi^{N_d}_{n=1} \sum_{z_{dn}}p(z_{dn}|\theta_d)p(w_{dn}|z_{dn},\beta) \bigg ) d\theta_d \ .
$$

The probability of the whole corpus is the given by the product over all documents of the above.

In this implementation of the LDA model we tread $K$ as a fixed hyper parameter. For the $\alpha$ and $\beta$ priors we test symmetric and asymmetric settings.

In [2]:
ntopics = 5

#due to size constraints we load the stan results where the samples have been averaged out (file with samples >200mb)
with open('stan_results.cpkl','r') as f:
    stan_data = cpkl.load(f)
    
with open('preprocessed_data.cpkl','r') as f:
    data = cpkl.load(f)
    
genes = data['genes']
t_train = data['t_train']
x_train = data['x_train']
words = data['words']
sampleid = data['sampleid']
    
theta_uni = stan_data['theta_uni']
phi_uni = stan_data['phi_uni']

theta_infor = stan_data['theta_infor']
phi_infor = stan_data['phi_infor']


## STAN Results
We fitted two different LDA models using stan.
 * Uniform prior: This models used a flat prior on the word distribution (alpha)
 * Informative prior: This models used a the background count frequencies as the prior for the count distribution (alpha)

#### Convergenve statistics

Due to long runtime the stan simulation was performed with 2000 samples and 4 chains. Below is seen a table of the convergene MCMC convergence statistics. (values on parenthesis af STD variation). The results suggests that

 * a) The effective number of samples (n_eff) is low and highly variable suggesting poor convergence
 * b) Rhat is relative high (not close to 1.0) suggeesting that more chains could be beneficial

| Parameter                   | N_eff   | Rhat   | 
|--------|----:|----:|
|Theta - Uniform prior        |44.2 (91.1) |1.5 |
|Phi - Uniform prior        |43.82 (152.8) |1.5 |
|Theta - informative prior     |21.1 (24.9) |1.5 |
|Phi - informative prior     |23.7 (87.9)  |1.5 |

Overall the STAN convergene statistics show that the models are not completely converged why the results might be misleading



In [ ]:
#STAN models (The models where run on servers due to high runtime (>10 hours))
LDA_model = """

data {
  int<lower=2> K;               // num topics
  int<lower=2> V;               // num words
  int<lower=1> M;               // num docs
  int<lower=1> N;               // total word instances
  int<lower=1,upper=V> w[N];    // word n
  int<lower=1,upper=M> doc[N];  // doc ID for word n
  vector<lower=0>[K] alpha;     // topic prior
  vector<lower=0>[V] beta;      // word prior
}
parameters {
  simplex[K] theta[M];   // topic dist for doc m
  simplex[V] phi[K];     // word dist for topic k
}
model {
  for (m in 1:M)  
    theta[m] ~ dirichlet(alpha);  // prior
  for (k in 1:K)  
    phi[k] ~ dirichlet(beta);     // prior
  for (n in 1:N) {
    real gamma[K];
    for (k in 1:K) 
      gamma[k] <- log(theta[doc[n],k]) + log(phi[k,w[n]]);
    increment_log_prob(log_sum_exp(gamma));  // likelihood
  }
}
"""


In [ ]:
#### STAN LDA topics

Below are seen heatmaps of the estimated expression for the five fitted topics of the STAN models. We see that

 * 1) The expression of a few genes seems to dominate the topics (noteably Hbb-b1).
 * 2) The topics appear relatively similar which suggests that the structure of the data have not been well captured

In [ ]:
data = pd.DataFrame(data=phi_uni.T,index=genes, columns=['Topic-1','Topic-2','Topic-3','Topic-4','Topic-5'])
cg = sns.clustermap(data,vmax=0.25,metric='euclidean',col_cluster=True,row_cluster=True)
out = plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
cg.ax_col_dendrogram.set_title('LDA Topics - Uniform Prior',size=25)

In [ ]:
data = pd.DataFrame(data=phi_infor.T,index=genes, columns=['Topic-1','Topic-2','Topic-3','Topic-4','Topic-5'])
cg = sns.clustermap(data,vmax=0.25,metric='euclidean',col_cluster=True,row_cluster=True)
out = plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
cg.ax_col_dendrogram.set_title('LDA Topics - Informative Prior',size=25)

## Conclusion

## References

Paul, Franziska, et al. "Transcriptional heterogeneity and lineage commitment in myeloid progenitors." Cell 163.7 (2015): 1663-1677 

Blei, David, et al. "Latent Dirichlet allocation." Journal of Machine Learning Research (2003): 993-1022